# Assignment 9: Final Project
## Computational Methods in Psychology (and Neuroscience)
### Psychology 4500/7559 --- Fall 2020


# Objectives

Upon completion of this assignment, students will have:

1. Described the list generation process in detail
2. Described the experiment details
3. Visualized processed data
4. Performed a statistical analysis to test the hypothesis

# Assignment

Write text (in MarkDown cells) and code (in Code cells) in a Jupyter notebook (after making a copy and renaming it to have your userid in the title --- e.g., A09_Final_Project_mst3k).


## Details

The goal of the final project is to synthesize material covered in the class and produce part of what would go into an actual scientific publication based on *one* of the experiments we ran in the class. Specifically, you will be writing part of the Methods and Results sections.

The basic template is below the code for loading and processing the data. There we outline what each section should include. As always, make sure to label all figures and be sure to refer to the code in the lesson notebooks as a guide for your analyses.

Please feel free to reach out to us on Slack if you have any questions along the way.

* ***When you are done, save this notebook as HTML (`File -> Download as -> HTML`) and upload it to the matching assignment on UVACollab.***  

## General Imports

In [6]:
# import some useful libraries
import numpy as np                # numerical analysis linear algebra
import pandas as pd               # efficient tables
import matplotlib.pyplot as plt   # plotting
import plotnine as pn 
import scipy.stats.distributions as dists     # probability distributions
from scipy import stats
from glob import glob
import os

from smile.log import log2dl

from ci_within import ci_within

## Custom SLOG loading function

In [7]:
# custom function to load slogs
def load_all_subj_logs(task_dir, log_file):
    # load in a list of all the subj
    subjs = [os.path.split(subj_dir)[-1] 
             for subj_dir in glob(os.path.join(task_dir, 's*'))]
    subjs.sort()

    # loop over subj and their data
    all_dat = []
    for subj in subjs:
        # set the file
        log_path = os.path.join(task_dir, subj, log_file)
        #print(log_path)

        # load the data
        all_dat.extend(log2dl(log_path, subj=subj))

    df = pd.DataFrame(all_dat)
    
    return df

## Load in all the data

In [8]:
# load the data from the word recog task
task_dir = os.path.join('..', 'lessons', 'data', 'Taskapalooza')

df_f = load_all_subj_logs(task_dir, 'log_flanker')
df_i = load_all_subj_logs(task_dir, 'log_image_test')
df_w = load_all_subj_logs(task_dir, 'log_word_test')
df_w.head()

,resp_map_lure,resp_map_target,block_num,trial_num,stim_on_time,stim_on_error,resp,resp_time_time,resp_time_error,rt,...,valence_sd,arousal_mean,arousal_sd,dominance_mean,dominance_sd,word_frequency,novelty,cond,subj,log_num
0,F,J,0,0,234.395511,0.0,J,235.284833,0.000180,0.889323,...,1.5700000000000001,5.3099999999999996,2.23,5.46,2.0499999999999998,3,target,neu,s001,0
1,F,J,0,1,235.885654,0.0,F,237.034670,0.000182,1.149016,...,1.5,4.1200000000000001,1.8300000000000001,5.6600000000000001,1.78,12,lure,neu,s001,0
2,F,J,0,2,237.616869,0.0,F,238.767406,0.000238,1.150537,...,1.8200000000000001,5.4500000000000002,2.1499999999999999,4.6399999999999997,2.0699999999999998,16,lure,neu,s001,0
3,F,J,0,3,239.624933,0.0,F,240.432295,0.000182,0.807362,...,1.24,3.9500000000000002,2.5800000000000001,5.3700000000000001,1.6399999999999999,19,lure,neu,s001,0
4,F,J,0,4,241.432209,0.0,F,242.545227,0.000192,1.113017,...,2.1600000000000001,3.6800000000000002,2.5699999999999998,5.8300000000000001,1.5,49,lure,neu,s001,0


## Some data clean-up

In [9]:
# it turns out the cond is easier to visualize as pure and mixed
def fix_conds(df, type_col):
    # loop over the unique subjects
    usubj = df.subj.unique()
    for s in usubj:
        # loop over their blocks
        ublocks = df.loc[df['subj']==s, 'block_num'].unique()
        for b in ublocks:
            # grab the data for that subj and block
            dfb = df.loc[(df['subj']==s)&(df['block_num']==b)]
            
            # get the unique types in that block
            uval = dfb[type_col].unique()
            if len(uval) > 1:
                # it's mixed
                df.loc[(df['subj']==s)&(df.block_num==b), 'cond'] = 'mixed'
            else:
                # it's the pure
                df.loc[(df['subj']==s)&(df.block_num==b), 'cond'] = 'pure'

# fix the conds in the recog experiments (updated in place)
fix_conds(df_i, type_col='in_out')
fix_conds(df_w, type_col='valence')

# add in log_rt columns
df_f['log_rt'] = np.log(df_f['rt'])
df_i['log_rt'] = np.log(df_i['rt'])
df_w['log_rt'] = np.log(df_w['rt'])

# must make correct an int
df_f['correct'] = df_f['correct'].astype(np.int)
df_i['correct'] = df_i['correct'].astype(np.int)
df_w['correct'] = df_w['correct'].astype(np.int)

# add in a column for whether they made an 'old' response
df_i['old_resp'] = (df_i['resp_map_target'] == df_i['resp']).astype(np.int)
df_w['old_resp'] = (df_w['resp_map_target'] == df_w['resp']).astype(np.int)

# process some of the valence info
df_w['valence_mean'] = df_w['valence_mean'].astype(np.float)
df_w['arousal_mean'] = df_w['arousal_mean'].astype(np.float)
df_w['dominance_mean'] = df_w['dominance_mean'].astype(np.float)
df_w['abs_valence'] = np.abs(df_w['valence_mean'] - 5.0)
df_w['abs_arousal'] = np.abs(df_w['arousal_mean'] - 5.0)

# Your text and code goes below here

*All code above should work without modification.*

# Hypothesis

*Fill in hypothesis (or hypotheses) here. You must pick something that will give rise to an analysis we did not perform in class, though it could be a similar analysis on a different dataset.*

# Methods

*This section should read like a methods section of a journal article. Fill in the two sub-sections below.*

## List generation

*Provide enough detail (in words) about the list generation so that someone could recreate the list generation code themselves. Be sure to state all the specific parameters used, including number of stimuli per block and number of blocks. Refer to the list generation code we provided to find all the information you need.*

## SMILE Experiment Details

*Provide enough detail so that someone could implement the experiment presentation and response collection, including all timing information and how the blocks were structured and presented.*


# Results

*In this section, state a specific question, then define your dependent and independent variables that will help you answer that question. As stated above, your question must give rise to an analysis that is not identical to one we performed in class (i.e., you must do more than copy and paste code with zero changes. That said, the analysis can match those from the class quite closely.*

## Data processing and visualization

*With the lessons as a guide, process your data to create the necessary data frame to plot the visualization associated with the question stated above. Then plot those data.*

In [ ]:
# Code for data processing and visualization

*Put text here describing what is in your plot like a detailed figure caption.*

## Statistical test and interpretation

*Perform a statistical test to support your conclusions with regard to your question outlined above. This can be with either statsmodels or with bambi.*

In [ ]:
# Code for statistical test (can be either with statsmodels or bambi)

*Put text here describing the results of your statistical test

# Discussion

***Graduate students only!!!***

*In one to two paragraphs do the following: a) Place the study in the larger literature, summarizing some of the similar work in the field and how this study compares, b) write some analysis of the findings from the study (even if they are null results) and then describe a follow-up study with a new variant of the experiment that you think might help answer further questions on the topic.